In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [35]:
stat = pd.read_csv('../keyhistory.csv') # data to read
stat["time"] = (pd.to_datetime(stat["time"], unit = 's'))

In [ ]:
pd.set_option('display.max_rows', 10)

In [ ]:
# TODO:
#
# интегральный график того сколько раз нажимал кнопку каждую
# типа например кнопка А - 8 числа 500 нажатий, затем 9 числа еще 500 нажатий, 
# на графике 8 числа - 500, 9 числа - 1к
# -------------------------------------
# pie-чарт с топом нажатых кнопок
# ------------------------------------
# логгинг мыши с перспективой получения heatmap кликов 

In [37]:
def AllTimeStrokes():
    """клавиши и соответствующее им количество нажатий за все время"""
    # аналогично stat.groupby(by='key').size()
    # topN значений - head(N), что такое tail(N) понятно
    return stat.key.value_counts()


def SplitStrokes(scale = "d", data = stat, allowEmpty = True):
    """число нажатий, разбитое на промежутки, данные в аргументе
    может считать нажатия отдельно взятой клавиши, см. аргумент data
    может убирать пустые вхождения, см. аргумент allowEmpty"""
    
    if (scale == "h" or scale == "H"): # час
        timing = "1H"
    elif (scale == "d" or scale == "D"): # день
        timing = "24H"
    elif (scale == "w" or scale == "W"): # неделя
        timing = "1W"
    elif (scale == "m" or scale == "M"): # месяц
        timing = "1M"
    elif (scale == "y" or scale == "Y"): # год
        timing = "1A"
    else:
        return 0
    
    resampled = data.resample(timing, on='time').size()
    if (not allowEmpty):
        return resampled[resampled != 0]
    return resampled


def Average(period):
    """среднее число нажатий за данный период"""
    return period.mean()


def GetKeyLog(key):
    """нажатия определенной клавиши за всё время"""
    if key not in stat.key.drop_duplicates().tolist():
        return 0
    return stat.groupby(by='key').get_group(key)


def InPeriod(data, start, offset):
    """возвращает интервал данных начиная со старта, заканчивая через оффсет времени"""
    # объяснение, зачем обрезать последнюю запись: 
    # 01:00 соответствует интервалу с 01:00 до 02:00, тогда если мы претендуем
    # дать данные за 2 часа, и мы начинаем с 3 часов ночи то мы должны давать 
    # записи 03:00 и 04:00 (это будет интервал 03:00-04:00 и 04:00-05:00, т.е 2 часа)
    # но пандас по умолчанию при индексировании включает как первую, так и последнюю
    # запись. то есть, взяв 03:00 и 2 hours, мы бы получили записи 03:00, 04:00 и 05:00
    # а это уже не 2 часа, а 3
    start_date = pd.Timestamp(start)
    end_date = start_date + pd.Timedelta(offset)
    return data[start_date : end_date].iloc[:-1] # обрезаем последнюю запись
   

In [44]:
print("Среднее число нажатий любых клавиш за час за всё время:",
      Average(SplitStrokes("h")))
print("Среднее число нажатий клавиши A за час за всё время:",
     Average(SplitStrokes('h', GetKeyLog("A"))))
print("Число нажатий клавиши A, разбитое по часам:\n",
     SplitStrokes('h', GetKeyLog("A")), "\n")
print("Число нажатий клавиши A, разбитое по часам и без пустых интервалов:\n",
     SplitStrokes('h', GetKeyLog("A"), False), "\n")
print("Среднее число нажатий любых клавиш за 2 часа 12/08/20 между 11 и 13 часами:",
     Average(InPeriod(SplitStrokes('h'), "2020-08-12 11:00:00", "2 hours")))

Среднее число нажатий любых клавиш за час за всё время: 739.75
Среднее число нажатий клавиши A за час за всё время: 20.5
Число нажатий клавиши A, разбитое по часам:
 time
2020-08-12 10:00:00     3
2020-08-12 11:00:00     0
2020-08-12 12:00:00    62
2020-08-12 13:00:00    17
Freq: H, dtype: int64 

Число нажатий клавиши A, разбитое по часам и без пустых интервалов:
 time
2020-08-12 10:00:00     3
2020-08-12 12:00:00    62
2020-08-12 13:00:00    17
dtype: int64 

Среднее число нажатий любых клавиш за 2 часа 12/08/20 между 11 и 13 часами: 967.0
